In [ ]:
import sys  
sys.path.insert(1, '/Users/asifahmed/Documents/Codes/MyRecourseProject')

from models.model_trainer import ModelTrainer
from evaluation.evaluator import Evaluator
from visualization.visualizer import Visualizer
from data_handling.crdit_data import Credit
from explainability.explainer_factory import get_explainer
import pandas as pd
from data_handling.dataset import Dataset

In [ ]:
data_instance = Dataset(target_column='D')
data_instance.load_csv('/Users/asifahmed/Documents/Codes/MyRecourseProject/synthetic_data/out/R1.csv')
data_instance.encode_categorical_columns()

In [ ]:
data_instance.print_features_names()

In [ ]:
data_instance.remove_outliers()

In [ ]:
data_instance.balanced_sample(300)

In [ ]:
data_instance.select_features(['D', 'age', 'X2'])
# data_instance.filter_non_zero()


In [ ]:
data_instance.print_duplicate_rows_excluding_columns(exclude_columns='D')

In [ ]:
data_instance.merge_duplicates_assign_majority(exclude_columns='A14')

In [ ]:
print((data_instance.data['A14']==1).sum())


In [ ]:
data_instance.print_numerical_columns()

In [ ]:
data_instance.plot_histogram()

In [ ]:
print(data_instance.data.dtypes)

In [ ]:
trainer = ModelTrainer()
X_train, X_test, y_train, y_test = trainer.split_data(data_instance.data, target_column='D')
X_train_scaled, X_test_scaled = trainer.scale_features(X_train, X_test)
model = trainer.train('svm', X_train_scaled, y_train)

# data = trainer.normalize_columns(data=data_instance.data, exclude_columns='D')

# trainer.split_data(data, target_column='D')


# model = trainer.train('svm')

evaluator = Evaluator(model, X_test_scaled, y_test)
evaluator.report()

In [ ]:
visualizer = Visualizer(model, trainer.X_test, trainer.y_test)
feature_names = ('Age', 'Resting Blood Pressure')
visualizer.plot_decision_boundary(feature_names)

In [ ]:
visualizer = Visualizer(model, X_test_scaled, y_test, X_original=X_test)
visualizer.plot_decision_boundary_interactive()

In [ ]:
print(X_test_scaled)

In [ ]:
print(X_test)

In [ ]:
import plotly.express as px

# Sample data
df = px.data.iris()

# Create an interactive scatter plot
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species", size='petal_length', hover_data=['petal_width'])

# Show the plot
fig.show()

In [1]:
import sys  
sys.path.insert(1, '/Users/asifahmed/Documents/Codes/MyRecourseProject')

from models.model_trainer import ModelTrainer
from evaluation.evaluator import Evaluator
from visualization.visualizer import Visualizer
from data_handling.crdit_data import Credit
from explainability.explainer_factory import get_explainer
import pandas as pd
from data_handling.dataset import Dataset


data_instance = Dataset(target_column='D')
data_instance.load_csv('/Users/asifahmed/Documents/Codes/MyRecourseProject/synthetic_data/out/R1.csv')
data_instance.encode_categorical_columns()
data_instance.remove_outliers()
data_instance.balanced_sample(300)
data_instance.select_features(['D', 'age', 'X2'])

trainer = ModelTrainer()
X_train, X_test, y_train, y_test = trainer.split_data(data_instance.data, target_column='D')
X_train_scaled, X_test_scaled = trainer.scale_features(X_train, X_test)
model = trainer.train('svm', X_train_scaled, y_train)
evaluator = Evaluator(model, X_test_scaled, y_test)
evaluator.report()

feature_names = ('Age', 'X2')
visualizer = Visualizer(model, X_test_scaled, y_test, X_original=X_test)
visualizer.plot_decision_boundary_interactive(feature_names=feature_names)

CSV file loaded successfully with delimiter: ','
Encoded G
Encoded job
Encoded D
Removed 70 outliers. New dataset size: 4930
Selected features are now active: ['D', 'age', 'X2']
Data split into train and test sets.
svm model trained successfully.
Accuracy: 0.5416666666666666
Precision: 0.55
Recall: 0.5409836065573771
F1 Score: 0.5454545454545454
Confusion Matrix:
 [[32 27]
 [28 33]]
Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.54      0.54        59
           1       0.55      0.54      0.55        61

    accuracy                           0.54       120
   macro avg       0.54      0.54      0.54       120
weighted avg       0.54      0.54      0.54       120



In [ ]:
data_instance.print_continuous_columns()

In [ ]:
# Generate explanations using DICE
continuous_features = ['A2', 'A14']
explainer = get_explainer('dice', model=model, data=data_instance.data,
                          continuous_features=continuous_features,
                          outcome_name='A15')
query_instance = trainer.X_test.iloc[0:5]  # Selecting the first test instance
counterfactuals = explainer.generate_explanation(query_instance, total_CFs=5)